In [1]:
import sys
sys.path.insert(0, '/tf/utils/')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pesq import pesq
import pystoi
from scipy.signal import lfilter, wiener
import scipy.linalg 
from tqdm import tqdm

from utils import load_wav, calculate_snr, itakura_distortion, performance, preemphasis, vad
from sound import Sound
from batch import DataGenerator

from multiprocessing import Pool, cpu_count

In [3]:
base_shape_size = 8000

In [4]:
# carrega sons de ruido e sons de voz
sound_base = Sound('../Dados/Base/', '../Dados/ESC-50-master/audio/', base_shape_size)

Loading noise files: 100%|██████████| 2000/2000 [00:09<00:00, 221.98it/s]


In [5]:
# Parâmetros do sistema
order = 16  # Ordem da análise LPC
window_size = 250  # Tamanho da janela
sample_rate = 8000  # Taxa de amostragem (exemplo)

In [6]:
data_generator = DataGenerator(sound_base.train_X, sound_base.noise_sounds)

In [7]:
# Adicionar filtro de pré-ênfase no sinal para obter valores mais adequados de coeficientes LPC

def lpc_analysis(signal, order):
    # Aplicar janelamento de Hamming
    windowed_signal = signal * np.hamming(len(signal))

    # Calcular os coeficientes LPC usando autocoeficientes
    autocorr = np.correlate(windowed_signal, windowed_signal, mode='full')
    r = autocorr[len(autocorr) // 2:len(autocorr) // 2 + order + 1]
    
    # Usar a função toeplitz para realizar a decomposição de Levinson-Durbin
    A = scipy.linalg.solve_toeplitz((r[:-1], r[1:]), -r[1:])
    
    # Calcular o erro de predição linear (residual)
    residual = signal - scipy.signal.lfilter(np.hstack([1, -A]), 1, signal)
    
    # Calcular a variância do erro de predição linear (residual)
    var_residual = np.var(residual)

    return A, np.sqrt(var_residual), r

In [8]:
def build_matrices(A, window_size):
    Ak = np.zeros((order, order))
    Ak[:, 0] = -A
    Ak[:-1, 1:] = np.eye(order - 1)

    H = np.zeros((1, order))
    H[0, 0] = 1.0

    return Ak, H

In [9]:
def kalman_filter(signal, Ak, H, Q, R):
    x_hat = np.zeros(order)  # Estado estimado
    P = np.eye(order)  # Covariância estimada

    filtered_signal = []

    for sample in signal:
        # Atualização temporal (Predição)
        x_hat = np.dot(Ak, x_hat)
        P = np.dot(np.dot(Ak, P), Ak.T) + Q

        # Atualização de mensuração (Correção)
        K = np.dot(np.dot(P, H.T), np.linalg.inv(np.dot(np.dot(H, P), H.T) + R))
        x_hat = x_hat + np.dot(K, (sample - np.dot(H, x_hat)))
        P = P - np.dot(np.dot(K, H), P)

        filtered_signal.append(x_hat[0])  # Apenas a primeira componente é o sinal estimado

    return np.array(filtered_signal)

In [10]:
def process_signal(signal, window_size, order, sample_rate, SNR_dB=10.):
    filtered_signal = []

    for i in range(0, len(signal), window_size):
        window_samples = signal[i:i+window_size]

        # Aplica o filtro de pré-ênfase
        emphasized_signal = preemphasis(window_samples, alpha=0.97)

        # Aplica a detecção de fala no sinal
        is_voice = vad(emphasized_signal, sample_rate)
        # is_voice = True

        if is_voice:
            # Realizar análise LPC e construir as matrizes Ak e H
            A, sigma, _ = lpc_analysis(emphasized_signal, order)
            Ak, H = build_matrices(A, len(emphasized_signal))
    
            # Calcular a variância do erro de aquisição R com base no SNR linear
            SNR_linear = 10.**(SNR_dB / 10.)
            Rx = 1. / SNR_linear
            
            # Calcular Q e R (assumindo que não mudam dentro da janela)
            Q = np.eye(order) * sigma  # Variância do erro de predição
            R = np.eye(1) * Rx  # Variância do erro de aquisição
    
            # Aplicar o filtro de Kalman na janela
            filtered_window = kalman_filter(window_samples, Ak, H, Q, R)
            filtered_signal.extend(filtered_window)

        else:
            window_size = 5
            window = np.ones(window_size) / window_size
            
            # Aplica o filtro de média móvel
            smoothed_signal = np.convolve(emphasized_signal, window, mode='same')
            
            # Certifique-se de que o sinal suavizado tenha o mesmo tamanho do sinal de entrada
            if len(smoothed_signal) < len(emphasized_signal):
                # Se o sinal suavizado for menor, adicione zeros à direita para igualar o tamanho
                shortfall = len(emphasized_signal) - len(smoothed_signal)
                smoothed_signal = np.append(smoothed_signal, np.zeros(shortfall))
            elif len(smoothed_signal) > len(emphasized_signal):
                # Se o sinal suavizado for maior, corte o excesso à direita
                smoothed_signal = smoothed_signal[:len(emphasized_signal)]
            
            # Adicione o sinal suavizado filtrado ao resultado final
            filtered_signal.extend(smoothed_signal)

    # print(len(filtered_signal))

    return np.array(filtered_signal)

In [11]:
def process_batch(args):
    x_batch, y_batch, SNR_dB_batch, window_size, order, sample_rate = args
    filtered_batch = [process_signal(noisy_signal, window_size, order, sample_rate, SNR_dB=SNR_dB) for noisy_signal, SNR_dB in zip(x_batch, SNR_dB_batch)]
    
    pesq_scores = [pesq(8000, clean, filtered.reshape(-1), 'nb') for clean, filtered in zip(y_batch, filtered_batch)]
    stoi_scores = [pystoi.stoi(clean, filtered, 8000) for clean, filtered in zip(y_batch, filtered_batch)]
    snr_scores = [calculate_snr(clean, filtered) for clean, filtered in zip(y_batch, filtered_batch)]
    ID_scores = [itakura_distortion(clean, filtered, window_size, order) for clean, filtered in zip(y_batch, filtered_batch)]
    
    return pesq_scores, stoi_scores, snr_scores, ID_scores

In [16]:
batch_num = 50
df_resultado = pd.DataFrame()

In [17]:
num_processes = cpu_count()  # Usar o número de núcleos da CPU
    
with Pool(processes=num_processes) as pool:
    results = []
    
    for _ in tqdm(range(batch_num)):
        x_batch, y_batch, metrics_batch_df = next(data_generator.generate_sample_metrics(window_size, order, batch_size=128))
        SNR_dB_batch = metrics_batch_df['SNR']
        
        args = (x_batch, y_batch, SNR_dB_batch, window_size, order, sample_rate)
        result = pool.apply_async(process_batch, (args,))
        results.append((result, metrics_batch_df))
    
    df_resultado = pd.DataFrame()
    
    for result, metrics_batch_df in results:
        pesq_scores, stoi_scores, snr_scores, ID_scores = result.get()
        metrics_batch_df['PESQ (Filtered)'] = pesq_scores
        metrics_batch_df['STOI (Filtered)'] = stoi_scores
        metrics_batch_df['SNR (Filtered)'] = snr_scores
        metrics_batch_df['ID (Filtered)'] = ID_scores
        
        df_resultado = pd.concat([df_resultado, metrics_batch_df], ignore_index=True)

100%|██████████| 50/50 [08:51<00:00, 10.62s/it]


Exception: x and y should have the same length,found (8000,) and (6530,)

In [ ]:
df_resultado.describe()

In [ ]:
performance(df_resultado, 'Kalman')